In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.keras as keras


In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state = 42)

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
X_train.shape

(11610, 8)

In [5]:
def huber_fn(y_true, y_pred):
    error = y_true-y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [6]:
input_shape = X_train.shape[1:]

net = keras.models.Sequential([
    keras.layers.Dense(30, activation=keras.activations.selu, kernel_initializer=keras.initializers.lecun_normal, input_shape = input_shape),
    keras.layers.Dense(1)
])

In [7]:
net.compile(loss = huber_fn, optimizer=keras.optimizers.Nadam(), metrics=['mae'])
net.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled,y_valid), epochs= 2, workers=0, )

Epoch 1/2
363/363 [==============================] - 1s 3ms/step - loss: 0.5832 - mae: 0.9328 - val_loss: 0.2128 - val_mae: 0.5076
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2120 - mae: 0.5084 - val_loss: 0.2040 - val_mae: 0.4917
